In [16]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['correlate']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import statsmodels.api as sm
from scipy.signal import find_peaks

In [36]:
import pandas as pd
from scipy.stats import pearsonr 
import numpy as np 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from scipy.io import wavfile
from scipy.signal import correlate
from matplotlib import rcParams
rcParams['figure.figsize'] = (128, 48)

In [6]:
path = "/content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010"

In [7]:
sound_files = sorted(glob(path+'/*.wav'))

In [8]:
fs = 22050
windowlength = 32
frameshift = 5
left_padding = 16
right_padding = 16

In [9]:
def autocorr_method(frame, sfreq, threshold=0.46, fmin=100, fmax=600):
    """Estimate pitch using autocorrelation
    """

    # Calculate autocorrelation using scipy correlate
    frame = frame.astype(np.float)
    frame -= frame.mean()
    amax = np.abs(frame).max()
    if amax > 0:
        frame /= amax
    else:
        return 0

    corr = correlate(frame, frame)
    # keep the positive part
    corr = corr[len(corr)//2:]

    # Find the first minimum
    dcorr = np.diff(corr)
    rmin = np.where(dcorr > 0)[0]
    if len(rmin) > 0:
        rmin1 = rmin[0]
    else:
        return 0

    # Find the next peak
    peak = np.argmax(corr[rmin1:]) + rmin1
    rmax = corr[peak]/corr[0]
    f0 = sfreq / peak

    if rmax > threshold and f0 >= fmin and f0 <= fmax:
        return f0, peak
    else:
        return 0, peak

In [10]:
if fs is not None:
  print('id,frequency', fs)
out_dict = {}
for filename in sound_files:
  print("Processing:", filename)
  sfreq, data = wavfile.read(filename)
  nsamples = len(data)
  # From miliseconds to samples
  ns_windowlength = int(round((windowlength * sfreq) / 1000))
  ns_frameshift = int(round((frameshift * sfreq) / 1000))
  ns_left_padding = int(round((left_padding * sfreq) / 1000))
  ns_right_padding = int(round((right_padding * sfreq) / 1000))
  pitch = []
  for ini in range(-ns_left_padding, nsamples - ns_windowlength + ns_right_padding + 1, ns_frameshift):
    first_sample = max(0, ini)
    last_sample = min(nsamples, ini + ns_windowlength)
    frame = data[first_sample:last_sample]
    f0, peak = autocorr_method(frame, sfreq)
    pitch.append((f0, peak + first_sample))
  
  #plot(data)
  #xlim(0, len(data))
  #title(sound_file.split('/').pop())

  pitch = [i for i in pitch if i[0] != 0]

  only_pitch = [i[0] for i in pitch]
  only_point = [i[1] for i in pitch]

  #plot(only_point, only_pitch, 'o')
  #title(filename.split('/').pop() + " Frequency")

  out_dict[filename.split('/').pop()] = pitch

id,frequency 22050
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0001.wav


<ipython-input-9-11d57903ab18>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  frame = frame.astype(np.float)


Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0002.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0003.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0004.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0005.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0006.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0006_speech.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0007.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0007_speech.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0008.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0009.wav
Processing: /content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0010.wav


In [13]:
out_dict

{'cta0001.wav': [(167.04545454545453, 132),
  (157.5, 227),
  (156.38297872340425, 338),
  (155.28169014084506, 449),
  (154.1958041958042, 560),
  (153.125, 671),
  (153.125, 781),
  (154.1958041958042, 890),
  (153.125, 1001),
  (154.1958041958042, 1110),
  (152.06896551724137, 1222),
  (152.06896551724137, 1332),
  (151.02739726027397, 1443),
  (152.06896551724137, 1552),
  (147.98657718120805, 1666),
  (147.98657718120805, 1886),
  (147.98657718120805, 2216),
  (147.98657718120805, 2326),
  (148.98648648648648, 2435),
  (152.06896551724137, 2542),
  (155.28169014084506, 2649),
  (164.55223880597015, 2861),
  (168.3206106870229, 2968),
  (182.23140495867767, 3068),
  (193.42105263157896, 3171),
  (200.45454545454547, 3277),
  (210.0, 3382),
  (222.72727272727272, 3486),
  (229.6875, 3593),
  (237.09677419354838, 3700),
  (247.75280898876406, 3806),
  (253.44827586206895, 3914),
  (262.5, 4021),
  (272.22222222222223, 4128),
  (279.11392405063293, 4236),
  (282.6923076923077, 4345),


In [14]:
filename

'/content/drive/MyDrive/STUDY_MY/Нейросети/cta0001-0010/cta0010.wav'

In [18]:
plot(data)
xlim(0, len(data))
plot(only_point, only_pitch, 'o')
title(filename.split('/').pop() + " Frequency")

Output hidden; open in https://colab.research.google.com to view.

In [19]:
def detect_encoding(file_path):
    encoding = "utf-8"
    try:
        l = open(file_path, 'r', encoding="utf-8").read()
        if l.startswith("\ufeff"):
            encoding = "utf-8-sig"
    except UnicodeDecodeError:
        try:
            open(file_path, 'r', encoding="utf-16").read()
            encoding = "utf-16"
        except UnicodeError:
            encoding = "cp1251"
    return encoding

In [20]:
def read_seg(path):
  with open(path, "r", encoding=detect_encoding(path)) as f:
    lines = [i.strip() for i in f.readlines()]
  return lines

In [21]:
seg_files = sorted(glob(path+'/*.seg_G1'))

In [22]:
last_seg_file = seg_files.pop()

In [34]:
true_points = [int(i.split(",")[0])/2 for i in read_seg(last_seg_file)[7:]][1:-1]

In [38]:
len(only_point)

871

In [40]:
len(true_points)

790

In [37]:
df = pd.DataFrame(dict(x=only_point))

CORR_VALS = np.array(true_points)
def get_correlation(vals):
    return pearsonr(vals, CORR_VALS)[0]

df['correlation'] = df.rolling(window=len(CORR_VALS)).apply(get_correlation)
df['correlation'].values 

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

Есть лишние значения, 81 штука. В остальном попадание к идеалу близко к 100%.
Стало быть можем считать, что точнось примерно равна 90% при учете лишних значений